In [215]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

import urllib3

urllib3.disable_warnings()

In [216]:
# # Single page set up
# url = f"https://scrapeme.live/shop/"
# response = requests.get(url, verify=False)
# df = bs(response.content, 'html.parser')

# itemList = df.find_all("li", {"class": "product"})
# len(itemList)

In [217]:
# Multi page set up
titles = []
for page in range(1, 2):
    url = f"https://scrapeme.live/shop/page/{page}"
    response = requests.get(url, verify=False)
    df = bs(response.content, 'html.parser')

    title = df.find_all("h2", {"class": "woocommerce-loop-product__title"})

    for t in title:
        titles.append(t.get_text())
titles

['Bulbasaur',
 'Ivysaur',
 'Venusaur',
 'Charmander',
 'Charmeleon',
 'Charizard',
 'Squirtle',
 'Wartortle',
 'Blastoise',
 'Caterpie',
 'Metapod',
 'Butterfree',
 'Weedle',
 'Kakuna',
 'Beedrill',
 'Pidgey']

In [218]:
dict = {
    "name": [],
    "price": [],
    "desc": [],
    "stock": [],
    "weight": [],
    "SKU": [],
    "tags": []
}
for t in titles:
    url = f"https://scrapeme.live/shop/{t}"
    resp = requests.get(url, verify=False)
    dr = bs(resp.content, 'html.parser')

    dict["name"].append(t)

    value = dr.find("p", {"class": "price"})
    value = value.text.strip()
    value = value.split("£")[1]
    dict["price"].append(float(value))

    descTag = dr.find("div", {"class": "woocommerce-product-details__short-description"})
    dsc = descTag.get_text(strip=True)
    dict["desc"].append(dsc)

    
    unitTag = dr.find("p", {"class":"stock in-stock"})
    units = unitTag.get_text(strip=True)
    units = units.split(" ")[0]
    dict["stock"].append(int(units))

    weightTag = dr.find("td", {"class" : "product_weight"})
    weight = weightTag.get_text(strip=True)
    weight = weight.split(" ")[0]
    dict["weight"].append(float(weight))


    idTag = dr.find("span",{"class": "sku"})
    id = idTag.get_text(strip=True)
    dict["SKU"].append(id)

    tagList = dr.find("span", {"class": "tagged_as"})
    tag = tagList.get_text()
    tag = tag.replace("\n", "").replace("Tags:", "").strip()
    dict["tags"].append(tag)

In [219]:
poke = pd.DataFrame(dict)
poke.to_csv("products.csv", index=False)
poke

,name,price,desc,stock,weight,SKU,tags
0,Bulbasaur,63.0,Bulbasaur can be seen napping in bright sunlig...,45,15.2,4391,"bulbasaur, Overgrow, Seed"
1,Ivysaur,87.0,There is a bud on this Pokémon’s back. To supp...,142,28.7,7227,"ivysaur, Overgrow, Seed"
2,Venusaur,105.0,There is a large flower on Venusaur’s back. Th...,30,220.5,7036,"Overgrow, Seed, venusaur"
3,Charmander,48.0,The flame that burns at the tip of its tail is...,206,18.7,9086,"Blaze, charmander, Lizard"
4,Charmeleon,165.0,Charmeleon mercilessly destroys its foes using...,284,41.9,6565,"Blaze, charmeleon, Flame"
5,Charizard,156.0,Charizard flies around the sky in search of po...,31,199.5,8840,"Blaze, Charizard, Flame"
6,Squirtle,130.0,Squirtle’s shell is not merely used for protec...,178,19.8,6094,"squirtle, Tiny Turtle, Torrent"
7,Wartortle,123.0,"Its tail is large and covered with a rich, thi...",24,49.6,9659,"Torrent, Turtle, wartortle"
8,Blastoise,76.0,Blastoise has water spouts that protrude from ...,88,188.5,5212,"blastoise, Shellfish, Torrent"
9,Caterpie,73.0,Caterpie has a voracious appetite. It can devo...,40,6.4,7492,"caterpie, Shield Dust, Worm"


In [220]:
# type(poke["name"][0]) str
# type(poke["price"][0]) float
# type(poke["desc"][0]) str
# type(poke["stock"][0]) int64
# type(poke["weight"][0]) float64
# type(poke["SKU"][0]) int
# type(poke["tags"][0]) list

In [221]:
def price_format(value):
    f = float(value)
    formatted = f"{value:06.2f}" 
    return float(formatted)

poke["price"] = poke["price"].map(price_format)

In [222]:
def sku_format(value):
    formatted = value.zfill(4)
    formatted = int(formatted)
    return formatted

poke["SKU"] = poke["SKU"].map(sku_format)

In [223]:
def tag_format(value):
    tagArr = value.split(",")
    for t in range(len(tagArr)):
        tagArr[t] = tagArr[t].strip()
    
    return tagArr

poke["tags"] = poke["tags"].map(tag_format)

In [ ]:
## DISCOUNTED PRICE EXTRACTION
## AVERAGE WEIGHT CALC

## WORD REPEAT ANALYSIS FROM DESCRIPTION
#~~~~~ WORD RECOGNITION FUNCTION ~~~~~~#
## PRICE VS WEIGHT ANALYSIS GRAPH
## PRICE VS STOCK ANALYSIS GRAPH
## TAG BASED ANALYSIS GRAPH

In [225]:
def discounted_price(price):
    discount = float(price) * 0.8
    return discount #float(f"{discount:06.2f}")

poke["discounted_price"] = poke["price"].map(discounted_price)        

In [ ]:
def average(value):
    metrics = {
        "mean": [],
        "median": [],
        "range": []
    }

    mean = round(float(value.mean()),2)
    median = round(float(value.median()),2)
    range = round(float(value.max() - value.min()),2)

    metrics["mean"].append(round(mean,2))
    metrics["median"].append((median))
    metrics["range"].append(range)

    return metrics
    
priceMetrics = average(poke["price"])
weightMetrics = average(poke["weight"])
stockMetrics = average(poke["stock"])

priceMetrics, weightMetrics, stockMetrics

({'mean': [114.75], 'median': [126.5], 'range': [143.0]},
 {'mean': [61.2], 'median': [25.35], 'range': [216.5]},
 {'mean': [138.75], 'median': [133.5], 'range': [261.0]})